In [9]:
import numpy as np
import cvxpy
import lab

In [3]:
## This is a basic CVXPY based implementation on a toy dataset for the paper 
## "Neural Networks are Convex Regularizers: Exact Polynomial-time Convex Optimization Formulations for Two-layer Networks"
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt


def relu(x):
    return np.maximum(0,x)
def drelu(x):
    return x>=0
n=10
d=3
X=np.random.randn(n,d-1)
X=np.append(X,np.ones((n,1)),axis=1)

y=((np.linalg.norm(X[:,0:d-1],axis=1)>1)-0.5)*2
beta=1e-4


dmat=np.empty((n,0))

## Finite approximation of all possible sign patterns
for i in range(int(1e2)):
    u=np.random.randn(d,1)
    dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)

dmat=(np.unique(dmat,axis=1))


# Optimal CVX
m1=dmat.shape[1]
Uopt1=cp.Variable((d,m1))
Uopt2=cp.Variable((d,m1))

## Below we use hinge loss as a performance metric for binary classification
yopt1=cp.Parameter((n,1))
yopt2=cp.Parameter((n,1))
yopt1=cp.sum(cp.multiply(dmat,(X*Uopt1)),axis=1)
yopt2=cp.sum(cp.multiply(dmat,(X*Uopt2)),axis=1)
cost=cp.sum(cp.pos(1-cp.multiply(y,yopt1-yopt2)))/n+beta*(cp.mixed_norm(Uopt1.T,2,1)+cp.mixed_norm(Uopt2.T,2,1))
constraints=[]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt1))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt2))>=0]
prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve()
cvx_opt=prob.value
print("Convex program objective value (eq (8)): ",cvx_opt)

Convex program objective value (eq (8)):  0.0007190913496571007


/opt/anaconda3/envs/convex_nn/lib/python3.8/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/opt/anaconda3/envs/convex_nn/lib/python3.8/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, Use

In [105]:
(X @ Uopt1.value).shape, dmat.shape

((10, 45), (10, 45))

In [113]:
U = np.sum(np.multiply(dmat, (X @ Uopt1.value)), axis=1)
V = np.sum(np.multiply(dmat, (X @ Uopt2.value)), axis=1)
(U - V).flatten() - y_hat.flatten()

array([-4.44089210e-16,  2.66453526e-15,  2.22044605e-16,  0.00000000e+00,
        1.55431223e-15,  2.22044605e-16,  4.44089210e-16,  0.00000000e+00,
       -4.44089210e-16,  4.44089210e-16])

In [60]:
def h(X, W1, W2):
    Z = X @ W1.T
    return np.max(Z, 0) @ W2.T

In [55]:
def relu_solution_mapping(weights, remove_sparse: bool = False):
    assert len(weights.shape) == 4

    weight_norms = (lab.sum(weights ** 2, axis=-1, keepdims=True)) ** (1 / 4)
    normalized_weights = lab.safe_divide(weights, weight_norms)

    num_classes = weights.shape[1]
    first_layer = None
    second_layer = []
    for c in range(num_classes):
        pre_zeros = [
            lab.zeros_like(weight_norms[0, c]) for i in range(2 * c)
        ]  # positive neurons
        post_zeros = [
            lab.zeros_like(weight_norms[0, c])
            for i in range(2 * (num_classes - c - 1))
        ]

        if first_layer is None:
            pre_weights = []
        else:
            pre_weights = [first_layer]

        first_layer = lab.concatenate(
            pre_weights
            + [
                normalized_weights[0][c],
                normalized_weights[1][c],
            ],
            axis=0,
        )

        w2 = lab.concatenate(
            pre_zeros
            + [
                weight_norms[0][c],
                -weight_norms[1][c],
            ]
            + post_zeros,
            axis=0,
        ).T
        second_layer.append(w2)

    second_layer = lab.concatenate(second_layer, axis=0)

    if remove_sparse:
        sparse_indices = lab.sum(first_layer, axis=1) != 0

        first_layer = first_layer[sparse_indices]
        second_layer = second_layer[:, sparse_indices]

    return first_layer, second_layer

In [56]:
weights = np.asarray([Uopt1.value.T[np.newaxis, ...], Uopt2.value.T[np.newaxis, ...]])
weights.shape

(2, 1, 45, 3)

In [57]:
first_layer, second_layer = relu_solution_mapping(weights)
first_layer.shape, second_layer.shape

((90, 3), (1, 90))

In [61]:
h(X, first_layer, second_layer)

array([7.01621409])

In [111]:
y_hat = np.maximum(X @ first_layer.T, 0) @ second_layer.T
y_hat

array([[ 1.        ],
       [ 3.69239712],
       [-1.        ],
       [-1.        ],
       [ 1.        ],
       [ 1.81576286],
       [ 2.80648925],
       [-1.        ],
       [ 1.        ],
       [ 1.        ]])

In [82]:
np.linalg.norm(y_hat - y) / 2

9.26761211956182

In [81]:
np.linalg.norm(first_layer, axis=1)

array([1.72471852e-05, 1.72464959e-05, 2.05872716e-05, 2.11135826e-05,
       1.60620196e-04, 2.15562773e-05, 2.19494285e-05, 1.60483635e-04,
       2.70245345e-05, 1.42338704e-05, 1.72468416e-05, 9.91689329e-01,
       9.91456341e-01, 1.91831752e-05, 6.09179436e-04, 1.11612755e+00,
       2.02131504e-03, 1.11522212e+00, 1.45998652e-05, 1.38891059e-05,
       6.45034297e-05, 1.31228200e-05, 1.26131698e-05, 1.73640856e-05,
       1.73350300e-05, 1.74425690e-05, 5.13641651e-01, 1.75074211e-05,
       1.03334763e-03, 5.12116138e-01, 1.10797982e+00, 4.44563645e-05,
       1.33105418e-05, 1.73580378e-05, 2.36083669e-05, 1.73954804e-05,
       7.46217802e-05, 7.41223669e-05, 1.72171970e-05, 1.66962748e-05,
       1.71625373e-05, 2.76290318e-05, 1.57590982e-05, 1.60637210e-05,
       1.24318090e-05, 1.72471852e-05, 1.72464959e-05, 1.67433679e-05,
       1.66045363e-05, 1.53505175e-05, 1.61630280e-05, 1.67846575e-05,
       1.45397462e-05, 2.36119061e-05, 2.66919709e-05, 1.72468417e-05,
      

In [116]:
np.max(1 - np.multiply(y.flatten(), y_hat.flatten())

array([-1.51517465e-09, -2.69239712e+00, -1.66286984e-11, -2.82847301e-10,
       -8.18489720e-11, -8.15762858e-01, -1.80648925e+00, -1.02677866e-11,
       -1.38005163e-11, -2.74509304e-11])